In [ ]:
import transformers

In [13]:
from datasets import load_dataset
import numpy as np
seed = 42
np.random.seed(seed)


dataset = load_dataset("roneneldan/TinyStories")
indices = np.random.randint(0, len(dataset["validation"]), size=10)
print(indices)

Repo card metadata block was not found. Setting CardData to empty.


[15795   860  5390 21575 11964 11284  6265 16850  4426 21962]


In [21]:
for i in indices:
    print(i)
    print(dataset["validation"][int(i)]["text"])

15795
Once upon a time, a naughty bear lived in a tree. He liked to eat yummy food, just like everyone else. One day, he had an idea. He suggested to the other animals that they should have a picnic. Everyone said it was a great idea, so they set off to find some food.

The bear found a big melon that he wanted to share out between them all. The other animals thought this was a great idea too.

But then, the naughty bear decided it was too big to carry. So he suggested they all eat it right away. All the animals laughed and started to eat the yummy melon. They enjoyed every bite. 

The bear was so happy that he suggested they do it again the next day!
860
Once there was an energetic girl called Amy. She was restless and always wanted to move. One day Amy's mom took her to the park. She made Amy do different exercises like running, jumping and hopping. Amy was excited about it and enjoyed every moment of it.

After a few hours, Amy got tired and wanted to take a break. Mom asked her to 

In [55]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

simple_165_model_path = "../chkpts/SlimPajama_Jan22/filtered/hidden_1024_num_layer_8_int_4096/final_model"
simple_tokenizer_path = "../tokenizer/SlimPajama/Jan22_15000"
simple_165_model = AutoModelForCausalLM.from_pretrained(simple_165_model_path)
simple_tokenizer = AutoTokenizer.from_pretrained(simple_tokenizer_path)

In [69]:
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-160m")
# model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m")
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m")
# model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-410m")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1b")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-1b")

In [70]:
import transformers
def prompt_and_completion(indices, ds, model, tokenizer, seed=42):
    transformers.set_seed(seed)
    prompts = []
    prompt_outputs = []

    for i in indices:
        prompt = dataset["validation"][int(i)]["text"]
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        #Generate completion
        output = model.generate(input_ids, max_new_tokens = 50, do_sample=True, top_p=0.9, pad_token_id=tokenizer.eos_token_id)
        # output = model.generate(input_ids, max_new_tokens = 2, pad_token_id=tokenizer.eos_token_id, num_beams  = 5, repetition_penalty=2.0)
        new_text = output[0][input_ids[0].shape[0]:]
        generated_text = tokenizer.decode(new_text, skip_special_tokens=True)
        prompts.append(prompt)
        prompt_outputs.append(generated_text)

    return prompts, prompt_outputs

In [71]:
# prompts, prompt_outputs = prompt_and_completion(indices, dataset, simple_165_model, simple_tokenizer)
prompts, prompt_outputs = prompt_and_completion(indices, dataset, model, tokenizer)

In [72]:
prompt_outputs

['\n\nSo, the next time, they set off.\n\nThe next time, the naughty bear decided that he had to eat the melon himself. So, he tried to squeeze it into his mouth. But, the others all laughed',
 ' Amy thought that it was too challenging for Amy and she wanted to rest for a few more minutes. Mom reminded Amy that the exercise could be done at any time. Amy was happy and agreed to let Mom take her home.\n\nOn the way',
 " The boy had promised the dragon that she would help her on her journey but the dragon was worried that she might end up in the hands of the dragon's mother. So, she made her intentions clear.\n\nThe dragon was happy to return home",
 '\n\nThe next day, Tom and Lily show their new house to their friends. They tell them about their new house and show it to some of their friends. They hope they will like it.\n\nTom says he can see his future home',
 " That night, she told her mom about the police officer who had saved her life.\n\nWhen the police officer had finished, Lily 

In [73]:
import pandas as pd

In [74]:
df = pd.DataFrame({"prompt": prompts, "output": prompt_outputs})

In [ ]:
csv_string = df.to_csv(sep='\t', index=False, header=False)
print(csv_string)

## ROC

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wza/roc_stories")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['storyid', 'storytitle', 'sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5'],
        num_rows: 98161
    })
})

In [3]:
import numpy as np
seed = 42
np.random.seed(seed)
sample_size = 100
smaller_dataset = dataset["train"].select(range(sample_size))
print(smaller_dataset)
indices = np.random.choice(len(smaller_dataset), size=20, replace=False)
print(indices)

Dataset({
    features: ['storyid', 'storytitle', 'sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5'],
    num_rows: 100
})
[83 53 70 45 44 39 22 80 10  0 18 30 73 33 90  4 76 77 12 31]


In [6]:
import pandas as pd
ao_childes_vocab = pd.read_csv('../data/AOChildes/AOChildes_word_frequency.csv')
vocab_words = set(ao_childes_vocab['word'])

In [37]:
def check_if_word_in_vocab(examples, vocab_words):
    # examples = examples["sentence1"] + " " + examples["sentence2"] + " " + examples["sentence3"] + " " + examples["sentence4"] + " " + examples["sentence5"]
    words = re.findall(r'\w+', examples)
    # words = examples["text"].split(" ")
    punctuation = ['.', ',', '!', '?', ':', ';', '..', '\n']
    out_of_vocab_words = []
    for word in words:
        if word.lower() not in vocab_words:
            out_of_vocab_words.append(word)
    # examples["out_of_vocab_words"] = out_of_vocab_words
    return out_of_vocab_words

In [35]:
for id in range(100):
    print(smaller_dataset[id])
    break

{'storyid': '9a51198e-96f1-42c3-b09d-a3e1e067d803', 'storytitle': 'Overweight Kid', 'sentence1': "Dan's parents were overweight.", 'sentence2': 'Dan was overweight as well.', 'sentence3': 'The doctors told his parents it was unhealthy.', 'sentence4': 'His parents understood and decided to make a change.', 'sentence5': 'They got themselves and Dan on a diet.'}


In [38]:
import re
for i in range(100):
    sent = smaller_dataset[i]["sentence1"] + " " + smaller_dataset[i]["sentence2"] + " " + smaller_dataset[i]["sentence3"] + " " + smaller_dataset[i]["sentence4"] + " " + smaller_dataset[i]["sentence5"]
    oov_words = check_if_word_in_vocab(sent, vocab_words)
    print(oov_words)

['unhealthy']
['Carrie', 'didn', 'Carrie', 'Carrie', 'gash']
['boyfriend', 'boyfriend', 'boyfriend', 'didn']
['barged', 'began', 'didn', 'coworker', 'intervened']
['text', 'hang']
['girlfriend']
['Dave', 'Bahamas', 'snorkeling', 'snorkeling', 'Dave', 'Dave']
[]
['widowed', 'mom', 'didn', 'mom', 'mom', 'mom']
['10', 'feet']
['Joshua']
['Hal', 'Hal', 'strained', 'Hal']
['Brenda', 'boyfriend', 'Brenda', 'Brenda']
['Yanice', 'fridge']
['Beethoven']
['2nd']
['mowers', 'blew', 'winnings', 'revoked', 'dui']
['miserably', 'homeless', 'alleyway', 'homeless']
['Saturday']
['reclined']
['Jessica']
['didn']
['competed', 'medal']
['Youtube', 'mom', 'mom', 'Youtube']
[]
['wasn', 'feet']
['couldn']
['Todd', 'Todd', 'Todd']
['Virgil', 'recliner', 'online', 'clashed', 'decor', 'Virgil', 'reupholstered', 'clashed', 'Virgil', 'recliner']
[]
['Joan', 'confessional', 'kneeled', 'confessing', 'parish', 'Joan', 'confessed', 'fantasized', 'Joan', 'confessional', 'parish']
['became']
[]
['Sadie']
['Gina', 'beg

In [4]:
simple_indices = [3, 6, 7, 8, 10, 11, 13, 15, 16, 22, 25, 26, 27, 28, 30, 33, 34, 36, 41, 42, 44, 46, 47, 51, 52, 57, 60, 62, 76, 77, 78, 87]

In [5]:
for i in simple_indices:
    # i-1 to offset the index
    print(smaller_dataset[int(i-1)])

{'storyid': '79b0da1f-e460-4173-ba58-8c9e2553c53a', 'storytitle': 'Beach', 'sentence1': 'Morgan enjoyed long walks on the beach.', 'sentence2': 'She and her boyfriend decided to go for a long walk.', 'sentence3': 'After walking for over a mile, something happened.', 'sentence4': 'Morgan decided to propose to her boyfriend.', 'sentence5': "Her boyfriend was upset he didn't propose to her first."}
{'storyid': '5d5e7aeb-332f-4c8b-a3c3-44585501e493', 'storytitle': 'Foolish Frank', 'sentence1': 'Frank had been drinking beer.', 'sentence2': 'He got a call from his girlfriend, asking where he was.', 'sentence3': 'Frank suddenly realized he had a date that night.', 'sentence4': 'Since Frank was already a bit drunk, he could not drive.', 'sentence5': 'Frank spent the rest of the night drinking more beers.'}
{'storyid': 'bdceebb0-c6e8-417f-b6ae-741a0a71fcd3', 'storytitle': 'Shark Adventure', 'sentence1': 'Dave was in the Bahamas on vacation.', 'sentence2': 'He decided to go snorkeling on his sec

In [7]:
small_dataest_indices = [i-1 for i in simple_indices]

In [10]:
filtered_ds = smaller_dataset.select(small_dataest_indices[0:15])

In [13]:
len(filtered_ds)

15

In [20]:
import transformers
def prompt_and_completion(ds, model, tokenizer, seed=42):
    transformers.set_seed(seed)
    prompts = []
    prompt_outputs = []

    for i in range(len(ds)):
        prompt = ds[int(i)]["sentence1"] + ds[int(i)]["sentence2"] + ds[int(i)]["sentence3"] + ds[int(i)]["sentence4"] + ds[int(i)]["sentence5"]
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        #Generate completion
        output = model.generate(input_ids, max_new_tokens = 50, do_sample=True, top_p=0.9, pad_token_id=tokenizer.eos_token_id)
        # output = model.generate(input_ids, max_new_tokens = 2, pad_token_id=tokenizer.eos_token_id, num_beams  = 5, repetition_penalty=2.0)
        new_text = output[0][input_ids[0].shape[0]:]
        generated_text = tokenizer.decode(new_text, skip_special_tokens=True)
        prompts.append(prompt)
        prompt_outputs.append(generated_text)

    return prompts, prompt_outputs

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

simple_165_model_path = "../chkpts/SlimPajama_Jan22/filtered/hidden_1024_num_layer_8_int_4096/final_model"
simple_tokenizer_path = "../tokenizer/SlimPajama/Jan22_15000"
simple_165_model = AutoModelForCausalLM.from_pretrained(simple_165_model_path)
simple_tokenizer = AutoTokenizer.from_pretrained(simple_tokenizer_path)

In [21]:
prompts, completions = prompt_and_completion(filtered_ds, simple_165_model, simple_tokenizer)

In [25]:
import  pandas as pd
df = pd.DataFrame({"prompt": prompts, "output": completions})
csv_string = df.to_csv(sep='\t', index=False, header=False)
print(csv_string)

Morgan enjoyed long walks on the beach.She and her boyfriend decided to go for a long walk.After walking for over a mile, something happened.Morgan decided to propose to her boyfriend.Her boyfriend was upset he didn't propose to her first.	He was upset because he knew she was not a good person. She was upset because he'd be upset. Morgan told her he'd be mad if she did not take him on. He did not want to leave her. She said no
Frank had been drinking beer.He got a call from his girlfriend, asking where he was.Frank suddenly realized he had a date that night.Since Frank was already a bit drunk, he could not drive.Frank spent the rest of the night drinking more beers.	He kept his eyes on the road and driving faster. Suddenly he realized that he was drunk. He decided to stop and try to calm himself down. He had a headache. The man took a beer and looked at Frank. He took a drink and
Dave was in the Bahamas on vacation.He decided to go snorkeling on his second day.While snorkeling, he saw 

In [28]:
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-160m")
# model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m")
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m")
# model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-410m")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1b")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-1b")
prompts, completions = prompt_and_completion(filtered_ds, model, tokenizer)

import  pandas as pd
df = pd.DataFrame({"prompt": prompts, "output": completions})
csv_string = df.to_csv(sep='\t', index=False, header=False)
print(csv_string)

Morgan enjoyed long walks on the beach.She and her boyfriend decided to go for a long walk.After walking for over a mile, something happened.Morgan decided to propose to her boyfriend.Her boyfriend was upset he didn't propose to her first.	The thing that made him upset was, after the long walk on the beach, they decided to take the short walk back.After walking about a mile, they started getting tired and decided to go back to the beach.After walking about a mile,
Frank had been drinking beer.He got a call from his girlfriend, asking where he was.Frank suddenly realized he had a date that night.Since Frank was already a bit drunk, he could not drive.Frank spent the rest of the night drinking more beers.	On the way home, Frank ran into a friend he knew.After Frank parked his car in front of the house, he and his friend saw a car in front of the house, blocking the driveway.This caused them to lose their license plates,
Dave was in the Bahamas on vacation.He decided to go snorkeling on h